<a href="https://colab.research.google.com/github/DepartmentOfStatisticsPUE/bi-2021/blob/main/materialy-wyklady/bi_2021_05_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Podwójnie odporne estymatory -- estymacja

W artykule *Chen, Y., Li, P., & Wu, C. (2020). Doubly Robust Inference With Nonprobability Survey Samples. Journal of the American Statistical Association, 115(532), 2011–2021. https://doi.org/10.1080/01621459.2019.1677241* przedstawiono dwa podejścia do estymacji (w przypadku badania na podstawie próby):

1. dwukrokowe (kolejność nie ma znaczenia):
  + estymacja propensity scores celem uzyskania wektora parametrów $\boldsymbol{\lambda}$
  + estymacja modelu $E(y_i|\boldsymbol{x}_i)$ w celu uzyskania wektora parametrów $\boldsymbol{\beta}$
  + wyznaczenie estymatora DR

\begin{equation}
\hat{\theta}_{\mathrm{DR}}=
\overbrace{
\frac{1}{\hat{N}^A} 
\sum_{i=1}^{n^A} 
\frac{
y_{i}-m\left(\hat{\boldsymbol{\beta}};\boldsymbol{x}_{i}\right)}
{\rho\left(\hat{\boldsymbol{\lambda}};\boldsymbol{x}_{i}\right)}}^{\text{Średnia ważona reszt z modelu}} 
+
\overbrace{
\frac{1}{\hat{N}^B} 
\sum_{i=1}^{n^B} 
m\left(\hat{\boldsymbol{\beta}}; \boldsymbol{x}_{i}\right)}^{\text{Średnia z predykcji dla całej populacji}},
\end{equation}


2. jednokrokowe, w którym jednocześnie rozwiązujemy następujący układ równań

\begin{equation}
\begin{array}{c}
\frac{1}{N} \sum_{i=1}^{N} R_{i}\left\{\frac{1}{\rho\left(\boldsymbol{\lambda};\boldsymbol{x}_{i}\right)}-1\right\}\left\{y_{i}-m\left(\boldsymbol{\beta};\boldsymbol{x}_{i}\right)\right\} \boldsymbol{x}_{i}=\mathbf{0} \\ 
\frac{1}{N} \sum_{i=1}^{N} \frac{R_{i}}{\rho\left(\boldsymbol{\lambda}; \boldsymbol{x}_{i}\right)} \dot{m}\left(\boldsymbol{\beta}; \boldsymbol{x}_{i}\right)-\frac{1}{N} \sum_{i \in \mathcal{S}_{\mathrm{B}}} d_{i}^{\mathrm{B}} \dot{m}\left(\boldsymbol{\beta}; \boldsymbol{x}_{i}\right)=\mathbf{0}\end{array},
\end{equation}

gdzie $\dot{m}\left(\boldsymbol{\beta}, \boldsymbol{x}\right) = \partial m(\boldsymbol{\beta}, \boldsymbol{x}) /\partial \boldsymbol{\beta}$, czyli w przypadku modelu liniowego: $\dot{m}\left(\boldsymbol{\beta}, \boldsymbol{x}\right)=\boldsymbol{x}$. 

Przy czym podejście to zakłada znajomość całej populacji, a estymator wariancji wyznaczony jest wyłącznie dla modelu regresji logistycznej.


## Implementacja w R 

### Propensity score 

**Na zajęciach skupimy się wyłącznie na pierwszym podejściu**. W pierwszej kolejności wyznaczymy $\rho(\boldsymbol{\lambda}; \boldsymbol{x}_i)$ rozwiązując następujący układ równań

$$
U(\boldsymbol{\lambda}) = \frac{\partial \log L^*(\boldsymbol{\lambda})}{\partial \boldsymbol{\lambda}} = \sum_{i \in S_A} \boldsymbol{x}_i - \sum_{i \in S_B} d_i^B \rho(\boldsymbol{\lambda}; \boldsymbol{x}_i)\boldsymbol{x}_i.
$$


który został wyprowadzony w [tym](https://github.com/DepartmentOfStatisticsPUE/bi-2021/blob/main/materialy-wyklady/bi_2021_04_19_chen_li_wu.ipynb) notatniku. Tam również można znaleźć poniższą funkcję:

```r
propensity_score_Chen_Li_Wu_grad <- function(lambda, x_a, x_b, d_b) {

  rho <- exp(x_b %*% lambda) / (1 + exp(x_b %*% lambda))
  
  ll_grad <- colSums(x_a) - colSums(d*rho*x_b)

  return(ll_grad)
}
```

Oznacza to, że:

1. musimy określić zmienne $x$,
2. napisać funkcję, która przyjmie następujące parametry: $\boldsymbol{\lambda}$, $\boldsymbol{x}$ oraz $d$,
3. musimy również określić punkty startowe tj. $\boldsymbol{\lambda}_0$. W tym celu skorzystamy z funkcji `glm`,
4. do rozwiązania tego układu potrzebujemy funkcji `multiroot` z pakietu `rootSolve`.

### Model $E(y|\boldsymbol{x})$

Na potrzeby zajęć przyjmiemy, że $E(y|\boldsymbol{x}) = m(\boldsymbol{\beta}; \boldsymbol{x})$ ma postać liniową:

$$
y_i = \boldsymbol{x}_i^T\boldsymbol{\beta}.
$$


## Komentarz

+ na zajęciach nie rozważamy kwestii doboru zmiennych do modelu,
+ w praktyce powinniśmy wcześniej wybrać zmienne,
+ w tym celu najlepiej wybrać metody oparte na cross walidacji (ang. cross-validation; CV) korzystając np. z LASSO czy SCAD.

## Instalacja pakietów oraz pobieranie danych

In [1]:
install.packages("rootSolve")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
library(rootSolve)
library(tidyverse)

Pobranie danych

In [3]:
download.file("https://raw.githubusercontent.com/DepartmentOfStatisticsPUE/bi-2021/main/data/popyt-zajecia-dane.csv",
              "popyt-dane.csv")
dane <- read.csv("popyt-dane.csv", stringsAsFactors = FALSE)
head(dane)              

,id_popyt,id_jednostki,waga,sek,klasa_pr,sekc_pkd,woj,zawod_kod2,wolne_miejsca,id_cbop,jedna_zmiana,wymiar_40,wolne_miejsca_cbop,wolne_miejsca_niepeln_cbop
,<int>,<chr>,<int>,<int>,<chr>,<chr>,<int>,<int>,<int>,<int>,<lgl>,<lgl>,<int>,<int>
1,1,a9cc990df6a99ab215a1bc13f51d4825c7d52d18,1,1,D,O,14,1,2,NA,NA,NA,NA,NA
2,2,a9cc990df6a99ab215a1bc13f51d4825c7d52d18,1,1,D,O,14,2,7,NA,NA,NA,NA,NA
3,3,c9dbaf50890165ebe810aa770de0e9df903dc35b,6,1,D,O,24,2,6,NA,NA,NA,NA,NA
4,4,718e0bba42bcec6ed98f9690db6d26cb7b93c880,1,1,D,R.S,14,2,7,NA,NA,NA,NA,NA
5,5,532a1879a692b9d7bbb7282ba757d028156ef341,1,1,D,R.S,14,2,6,NA,NA,NA,NA,NA
6,6,0b6b623fa45e257284a3049d097af322841337e3,1,1,D,R.S,22,2,1,NA,NA,NA,NA,NA


Przygotowanie danych do estymacji

In [4]:
proba_losowa <- dane %>%
  ungroup() %>%
  filter(!is.na(id_popyt)) %>%
  distinct(id_jednostki, waga, sek, wielk = klasa_pr, pkd = sekc_pkd, woj) 

proba_nielosowa <- dane %>%
  ungroup() %>%
  filter(!is.na(id_cbop)) %>%
  group_by(id_jednostki, sek, wielk = klasa_pr, pkd = sekc_pkd, woj) %>%
  summarise(odsetek = weighted.mean(jedna_zmiana, wolne_miejsca_cbop))


dane_estymacja <- bind_rows(
  proba_nielosowa %>% 
      ungroup()  %>%
      mutate(proba = 1, waga = 1, sek = factor(sek), woj = factor(woj)),
  proba_losowa %>% 
    ungroup()  %>%
    mutate(proba = 0, sek = factor(sek), woj = factor(woj))
)

head(dane_estymacja)

  

`summarise()` has grouped output by 'id_jednostki', 'sek', 'wielk', 'pkd'. You can override using the `.groups` argument.



id_jednostki,sek,wielk,pkd,woj,odsetek,proba,waga
<chr>,<fct>,<chr>,<chr>,<fct>,<dbl>,<dbl>,<dbl>
000167228537c46b1b084e9c501b065ca42374ce,1,M,Q,8,1,1,1
0001d7cc2ffede0b1bde90bd4441d63fda57bcf0,2,M,G,12,0,1,1
000be1bfb49ae8082ef18d325c2ca6a7aeb0efe6,2,M,C,22,0,1,1
000f0546b2f54d82de8efd89612c2cb9e8728112,2,S,P,12,1,1,1
001a4ae4b2153adce49fa5a77825990cbd008419,2,D,G,24,0,1,1
0022b78360fff23fb836b05a93cffafa1d951593,2,D,N,4,0,1,1


## Estymacja propenisty scores

Określamy punkty startowe wykorzystując funkcję `glm`. Ta funkcja posłuży również do określenia argumentów naszej funkcji tj. `x_a` i `x_b`.

In [5]:
m1 <- glm(proba ~ woj + wielk + sek + pkd, dane_estymacja, family = binomial, weights = waga)
summary(m1)


Call:
glm(formula = proba ~ woj + wielk + sek + pkd, family = binomial, 
    data = dane_estymacja, weights = waga)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-15.1136   -0.7098    1.5266    1.9084    2.7246  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept) -1.13143    0.07012 -16.135  < 2e-16 ***
woj4         0.59290    0.06493   9.131  < 2e-16 ***
woj6         0.22225    0.06806   3.265 0.001093 ** 
woj8         0.08578    0.08501   1.009 0.312927    
woj10       -0.10920    0.06162  -1.772 0.076382 .  
woj12       -0.44348    0.05809  -7.634 2.27e-14 ***
woj14       -0.66321    0.05202 -12.750  < 2e-16 ***
woj16        0.54342    0.07945   6.840 7.93e-12 ***
woj18        0.77734    0.06490  11.978  < 2e-16 ***
woj20        0.17500    0.08781   1.993 0.046274 *  
woj22       -0.01320    0.05762  -0.229 0.818800    
woj24       -0.29648    0.05255  -5.641 1.69e-08 ***
woj26        0.16736    0.08088   2.069 0.038522 *  
wo

Przygotowujemy dane do estymacji metodą PS

In [6]:
X <- model.matrix(m1) ## wyciągamy macierz X z obiektu m1
X_a <- X[dane_estymacja$proba == 1, ]
X_b <- X[dane_estymacja$proba == 0, ]
d_b <- dane_estymacja$waga[dane_estymacja$proba == 0] ## waga tylko dla tych rekordów z próby nielosowej
theta_0 <- coef(m1) ## parametry startowe
print(theta_0)

(Intercept)        woj4        woj6        woj8       woj10       woj12 
-1.13143144  0.59290290  0.22225067  0.08578418 -0.10919791 -0.44347549 
      woj14       woj16       woj18       woj20       woj22       woj24 
-0.66320954  0.54342174  0.77734499  0.17499654 -0.01320078 -0.29648025 
      woj26       woj28       woj30       woj32      wielkM      wielkS 
 0.16736498  0.41229717 -0.45433993  0.23647663 -0.74697007 -0.23289845 
       sek2      pkdD.E        pkdF        pkdG        pkdH        pkdI 
 0.05737559  0.50407033 -0.30484161 -0.26197458 -0.49829949  0.30314176 
       pkdJ      pkdK.L        pkdM        pkdN        pkdO        pkdP 
-1.20277711  0.06052943 -0.31623204  0.51655442 -0.47475660  0.76149170 
       pkdQ      pkdR.S 
 0.23515286  0.18382817 


Funkcja do wyznaczenia PS (gradient)

In [7]:
propensity_score_Chen_Li_Wu_grad <- function(lambda, x_a, x_b, d_b) {
  rho <- as.numeric(exp(x_b %*% lambda) / (1 + exp(x_b %*% lambda)))
  ll_grad <- colSums(x_a) - colSums(d_b*rho*x_b)
  return(ll_grad)
}

Rozwiązujemy układ równań $U(\boldsymbol{\lambda}) = \boldsymbol{0}$. Funkcja `multiroot` zwraca listę z następującymi argumentami 

+ `root` -- wektor parametrów $\hat{\boldsymbol{\lambda}}$, który spełnia równianie $U(\hat{\boldsymbol{\lambda}}) \approx \boldsymbol{0}$,
+ `f.root` -- jak blisko wartości zero jesteśmy,
+ `iter` -- po ilu iteracjach dochodzimy do celu

In [11]:
rsolve_res <- multiroot(f = propensity_score_Chen_Li_Wu_grad, 
                        start = theta_0, 
                        x_a = X_a, x_b = X_b, d_b = d_b)
print(rsolve_res)

$root
(Intercept)        woj4        woj6        woj8       woj10       woj12 
-0.65277121  0.83779550  0.19953188  0.10479761 -0.15756693 -0.60987005 
      woj14       woj16       woj18       woj20       woj22       woj24 
-0.84150300  0.76385615  1.17810496  0.22251032 -0.03753066 -0.40670702 
      woj26       woj28       woj30       woj32      wielkM      wielkS 
 0.20287478  0.57862243 -0.61021662  0.32742204 -1.02915884 -0.36412124 
       sek2      pkdD.E        pkdF        pkdG        pkdH        pkdI 
 0.05899377  0.77273741 -0.37783229 -0.33370352 -0.65174566  0.41179116 
       pkdJ      pkdK.L        pkdM        pkdN        pkdO        pkdP 
-1.42637011  0.06171389 -0.40677813  0.80034162 -0.69354695  1.25095429 
       pkdQ      pkdR.S 
 0.30286775  0.22227967 

$f.root
  (Intercept)          woj4          woj6          woj8         woj10 
-1.818989e-12 -2.273737e-13  5.684342e-14  0.000000e+00  1.136868e-13 
        woj12         woj14         woj16         woj18        

## Estymacja modelu E(y|x)

Zakładamy model liniowy

In [21]:
m2 <- lm(odsetek ~ sek + wielk + pkd + woj, data = dane_estymacja, subset = proba == 1)
summary(m2)


Call:
lm(formula = odsetek ~ sek + wielk + pkd + woj, data = dane_estymacja, 
    subset = proba == 1)

Residuals:
    Min      1Q  Median      3Q     Max 
-1.0605 -0.4442  0.1073  0.3299  0.8033 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept)  0.582102   0.025258  23.047  < 2e-16 ***
sek2        -0.123613   0.019737  -6.263 3.94e-10 ***
wielkM       0.284632   0.012309  23.125  < 2e-16 ***
wielkS       0.197199   0.011671  16.897  < 2e-16 ***
pkdD.E       0.057101   0.031656   1.804 0.071293 .  
pkdF         0.227868   0.015665  14.546  < 2e-16 ***
pkdG        -0.094976   0.013632  -6.967 3.46e-12 ***
pkdH        -0.127630   0.022239  -5.739 9.82e-09 ***
pkdI        -0.230242   0.020005 -11.509  < 2e-16 ***
pkdJ         0.164182   0.054519   3.011 0.002607 ** 
pkdK.L       0.181066   0.028248   6.410 1.53e-10 ***
pkdM         0.168925   0.025406   6.649 3.11e-11 ***
pkdN        -0.037535   0.022091  -1.699 0.089341 .  
pkdO         0.214979   0.03374

## Wyznaczamy estymator double robust

Część pierwsza równania

\begin{equation}
\frac{1}{\hat{N}^A} 
\sum_{i=1}^{n^A} 
\frac{
y_{i}-m\left(\hat{\boldsymbol{\beta}};\boldsymbol{x}_{i}\right)}
{\rho\left(\hat{\boldsymbol{\lambda}};\boldsymbol{x}_{i}\right)}
\end{equation}

In [28]:
nielosowa <- dane_estymacja %>% filter(proba == 1)

nielosowa <- nielosowa %>% 
    mutate(rho_Chen_Li_Wu = exp(as.numeric(X_a %*% rsolve_res$root))/(1 + exp(as.numeric(X_a %*% rsolve_res$root))),
           waga_Chen_Li_Wu = 1/rho_Chen_Li_Wu,
           reszty = resid(m2)) 

dr_part1 <- weighted.mean(nielosowa$reszty, nielosowa$waga_Chen_Li_Wu)
dr_part1

[1] -1.392226e-05

**Co oznacza powyższa wartość?**

Część druga równania

\begin{equation}
\frac{1}{\hat{N}^B} 
\sum_{i=1}^{n^B} 
m\left(\hat{\boldsymbol{\beta}}; \boldsymbol{x}_{i}\right)
\end{equation}

In [25]:
losowa <- dane_estymacja %>% 
    filter(proba == 0) %>%
    mutate(m_hat = predict(m2, subset(dane_estymacja, proba == 0)))

dr_part2 <- weighted.mean(losowa$m_hat, losowa$waga)
dr_part2

[1] 0.7172375

Razem i porównanie z estymatorem PS

In [33]:
dr <- dr_part1 + dr_part2
ps <- weighted.mean(nielosowa$odsetek, nielosowa$waga_Chen_Li_Wu)

print(
  c("0. Naive" = round(mean(nielosowa$odsetek)*100,2), 
    "1. Propenisty score" = round(ps*100,2), 
    "2. Podwójnie odporny" = round(dr*100, 2))
)

            0. Naive  1. Propenisty score 2. Podwójnie odporny 
               68.09                72.16                71.72 
